# Control Óptimo de Cohete 

<img src="YouWillNotGoToSpaceToday.png" width="350">

(Credito de Imagen a www.XKCD.com)

Se estudia el problema de Goddard, formulado por Robert H. Goddard en 1919 en "_A Method of Reaching Extreme Altitudes_". 

Consiste en definir el control óptimo de la potencia aplicada del motor de manera de lograr la máxima altura en el vuelo del cohete, tomando en cuenta el arrastre aerodinámico, el cambio en la masa del cohete y la variación en la fuerza de gravedad.

## Dinámica

La segunda ley de newton requiere que:

$$\frac{d}{dt}(m v) = \sum F_{ext}$$

Asumimos que el movimiento del cohete es solamente en la direccion radial al centro de la tierra (...hacia afuera...), la coordenada $h(t)$ mide la posición del cohete desde el centro de la tierra.

La velocidad es igual a $\dot{h}$, la masa del cohete es naturalmente variable $m(t)$ y las fuerzas aplicadas al cohete son la gravedad ($g(h)$) y el arrastre aerodinámico ($D(h,\dot{h})$).

Usando las hipótesis anteriores, la segunda ley de newton nos da:

$$ m \ddot{h} = -\dot{m} v_e -g(h)-D(h,\dot{h}) $$

El primer término se puede interpretar como el empuje producido por el motor, la pérdida de masa ($\dot{m}<0$) se corresponde con una fuerza aplicada en el sentido de la velocidad del cohete. Llamaremos a esa fuerza el Empuje (Thrust), la notaremos como $T$ y será nuestra variable de control.

Tenemos por lo tanto el sistema de ecuaciones:

$$\left\{ \begin{eqnarray}
m \ddot{h} &=& T -g(h)-D(h,\dot{h}) \\
\dot{m} &=& -\frac{T}{v_e}
\end{eqnarray}\right.
$$

La ley de gravedad es sencillamente:  $g(h) = g_0 (h_0/h)^2$

La definición del arrastre aerodinámico es: $D(h,\dot{h})=1/2 C_d A_d \left(\rho_0 e^{k(1-h/h_0)}\right) \dot{h}^2 $. Se toma en cuenta aquí una atmósfera que se atenúa de manera exponencial.

El sistema dinámico se puede adimensionar de manera de trabajar sin unidades. Por ejemplo: $\hat{h}=h/R$, siendo $R$ el radio de la tierra. Otro ejemplo: $\hat{m}=m/m_0$, siendo $m_0$ la masa inicial del cohete. La referencia optó por expresar el problema en forma adimensionada. Llevar los resultados a expresiones con dimensión requiere simplemente multiplicar los resultados por las constantes correctas.


## Comentario de Solución Mediante Principio del Mínimo de Poyntragin

Se puede ver claramente que el Hamiltoniano va a ser lineal en la variable de control $T$. Por lo tanto el principio del Mínimo lleva a estudiar las situaciones en las cuales el coeficiente que multiplica a $T$ es positivo o negativo (para obtener control tipo Bang-Bang) y también cuando es cero.

Cuando el factor que multiplica a $T$ es cero, la condicion de mínimo no depende de $T$ y debemos recurrir a las ecuaciones adjuntas para hallar cuál es el comportamiento de $T$ en esa circunstancia. Este tipo de soluciones de control se llaman **Arcos Singulares**.

En la solucion numérica mostramos un control óptimo de tipo _Bang-Arco Singular_ e inclusive Bang-Arco Singular-Bang_.

## Solución Numérica del Control Óptimo
Referencia a versión original del notebook: https://www.juliaopt.org/notebooks/JuMP-Rocket.html

In [ ]:
using JuMP, Ipopt

In [ ]:
# Create JuMP model, using Ipopt as the solver
mod = Model(with_optimizer(Ipopt.Optimizer,print_level=0))

# Constants
# Note that all parameters in the model have been normalized
# to be dimensionless. See the COPS3 paper for more info.
h_0 = 1    # Initial height
v_0 = 0    # Initial velocity
m_0 = 1    # Initial mass
g_0 = 1    # Gravity at the surface

# Parameters
T_c = 3.5  # Used for thrust
h_c = 500  # Used for drag 
v_c = 620  # Used for drag **VER 3 CASOS: 200,620,1000.
m_c = 0.6  # Fraction of initial mass left at end ** Ver 0.6 y CASO: 0.4

# Derived parameters
c     = 0.5*sqrt(g_0*h_0)  # Thrust-to-fuel mass
m_f   = m_c*m_0            # Final mass
D_c   = 0.5*v_c*m_0/g_0    # Drag scaling
T_max = T_c*g_0*m_0        # Maximum thrust

n = 1000                                    # Time steps
@variable(mod, Δt ≥ 0, start = 1/n)        # Time step
@NLexpression(mod,t_f, Δt*n)               # Time of flight

# State variables
@variable(mod, v[0:n] ≥ 0)            # Velocity
@variable(mod, h[0:n] ≥ h_0)          # Height
@variable(mod, m_f ≤ m[0:n] ≤ m_0)    # Mass

# Control: thrust
@variable(mod, 0 ≤ T[0:n] ≤ T_max)

# Objective: maximize altitude at end of time of flight
@objective(mod, Max, h[n])

# Initial conditions
@constraint(mod, v[0] == v_0)
@constraint(mod, h[0] == h_0)
@constraint(mod, m[0] == m_0)

# Forces
# Drag(h,v) = Dc v^2 exp( -hc * (h - h0) / h0 )
@NLexpression(mod,drag[j=0:n], D_c*(v[j]^2)*exp(-h_c*(h[j]-h_0)/h_0))
# Grav(h)   = go * (h0 / h)^2
@NLexpression(mod,grav[j=0:n], g_0*(h_0/h[j])^2)

# Dynamics
for j in 1:n
    # h' = v
    # Rectangular integration
    # @NLconstraint(mod, h[j] == h[j-1] + Δt*v[j-1])
    # Trapezoidal integration
    @NLconstraint(mod,
        h[j] == h[j-1] + 0.5*Δt*(v[j]+v[j-1]))

    # v' = (T-D(h,v))/m - g(h)
    # Rectangular integration
    # @NLconstraint(mod, v[j] == v[j-1] + Δt*(
    #                 (T[j-1] - drag[j-1])/m[j-1] - grav[j-1]))
    # Trapezoidal integration
    @NLconstraint(mod,
        v[j] == v[j-1] + 0.5*Δt*(
            (T[j  ] - drag[j  ] - m[j  ]*grav[j  ])/m[j  ] +
            (T[j-1] - drag[j-1] - m[j-1]*grav[j-1])/m[j-1] ))

    # m' = -T/c
    # Rectangular integration
    # @addNLConstraint(mod, m[j] == m[j-1] - Δt*T[j-1]/c)
    # Trapezoidal integration
    @NLconstraint(mod,
        m[j] == m[j-1] - 0.5*Δt*(T[j] + T[j-1])/c)
end

# Provide starting solution
for k in 0:n
    set_start_value(h[k], 1)
    set_start_value(v[k], (k/n)*(1 - (k/n)))
    set_start_value(m[k], (m_f - m_0)*(k/n) + m_0)
    set_start_value(T[k], T_max/2)
end

# Print the optimization problem (mod)
#print(mod)
println("")

# Solve for the control and state
println("Solving...")
status = optimize!(mod)

# Display results
println("Solver status: ", termination_status(mod))
println("Max height: ", objective_value(mod))

## Presentación de los resultados numéricos

In [ ]:
using Gadfly

In [ ]:
# Determino las constantes para volver a tener dimensiones de [km] [seg]:

R_t = 6371 # radio de la tierra [km]
G_0 = 9.81 # aceleracion gravitatoria a nivel del mar [m/s^2]
c_t = sqrt(R_t*1000/G_0) # constante adimensionado para el tiempo [seg]

In [ ]:
tiempo = collect((0:n)*value(Δt)*c_t)

h_plot = plot(x=tiempo,y=collect(value.(h*R_t)), Geom.line,
                Guide.xlabel("Time (s)"), Guide.ylabel("Altitude [km]"))
m_plot = plot(x=tiempo,y=collect(value.(m)), Geom.line,
                Guide.xlabel("Time (s)"), Guide.ylabel("Mass"))
v_plot = plot(x=tiempo,y=collect(value.(v)*R_t/c_t), Geom.line,
                Guide.xlabel("Time (s)"), Guide.ylabel("Velocity (km/s)"))
T_plot = plot(x=tiempo,y=collect(value.(T)), Geom.line,
                Guide.xlabel("Time (s)"), Guide.ylabel("Thrust"))
d_plot = plot(x=tiempo,y=collect(value.(drag)), Geom.line,
                Guide.xlabel("Time (s)"), Guide.ylabel("Drag"))
f_plot = plot(x=collect(value.(h)*R_t),y=collect(value.(v*R_t/c_t)), Geom.line,
                Guide.xlabel("Altitude [km]"), Guide.ylabel("Velocity [km/s]"))
draw(SVG(6inch, 9inch), vstack( hstack(h_plot,m_plot),
                                hstack(v_plot,T_plot),
                                hstack(d_plot,f_plot)))